We'll look at how transformations and actions work and how they affect the execution of code. We'll also explore a wider range of the functions built into the Spark core.

The file hamlet.txt contains the entire text of Shakespeare's play Hamlet. Shakespeare is well-known for his unique writing style and is arguably one of the most influential writers in history. Hamlet is one of his most popular plays.

Let's perform some text analysis on it. The file is in pure text format, so it's not ready for analysis. Before we can proceed, we'll have to clean up and reformat the data.

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark

In [3]:
from pyspark import SparkContext

In [4]:
sc = pyspark.SparkContext()

raw_hamlet = sc.textFile('data/hamlet.txt')
raw_hamlet.take(5)

['hamlet@0\t\tHAMLET',
 'hamlet@8',
 'hamlet@9',
 'hamlet@10\t\tDRAMATIS PERSONAE',
 'hamlet@29']

The text file uses the tab character (\t) as a delimiter. We'll need to **split the file on the tab delimiter** and convert the results into an RDD that's more manageable.

In [7]:
split_hamlet = raw_hamlet.map(lambda line: line.split('\t'))
split_hamlet.take(5)

[['hamlet@0', '', 'HAMLET'],
 ['hamlet@8'],
 ['hamlet@9'],
 ['hamlet@10', '', 'DRAMATIS PERSONAE'],
 ['hamlet@29']]

Lambda functions are great for writing quick functions we can pass into PySpark methods with simple logic. They aren't ideal when we need to write more customized logic, however. Thankfully, PySpark lets us define a function in Python first, then pass it in. Any function that returns a sequence of data in PySpark (versus a guaranteed Boolean value, like filter() requires) must use a yield statement to specify the values to pull later.

'yield' is a Python technique that allows the interpreter to generate data as they work and pull it when necessary — instead of storing it to memory immediately. Because of its unique architecture, Spark uses this technique to reduce overhead and improve computation speed.

Spark runs the named function on every element in the RDD and restricts its scope. Each instance of the function can only access the object(s) you pass into the function — and the Python libraries available in your environment. If you try to refer to variables outside the scope of the function, or if you try to import libraries, the computation may crash. That's because Spark compiles the function's code to Java to run on the RDD objects (which are also in Java).

Finally, not all functions require yield. Only functions that generate a custom sequence of data do. For map() or filter(), we use return to return a value for every single element in the RDD we're running the functions on.

## flatMap()

flatMap() is different than map() because it doesn't require an output for every element in the RDD. The flatMap() method is useful whenever we want to generate a sequence of values from an RDD.

We'll use the flatMap() method with the named function hamlet_speaks to check whether a line in the play contains the text HAMLET in all caps (indicating that Hamlet is speaking).

We want an RDD object that contains tuples of the unique line IDs and the text "hamlet speaketh!," but only for the elements in the RDD that have "HAMLET" in one of the values. We can't use the map() method for this because it requires a return value for every element in the RDD.

We want each element in the resulting RDD to have the following format:

1. The first value should be the unique line ID (e.g.'hamlet@0'), which is the first value in each of the elements in the split_hamlet RDD.

2. The second value should be the string "hamlet speaketh!"

In [ ]:
def hamlet_spraks